## Method4 (Basically Method2 but without bbox)

In [1]:
from utils import *

#### Load a file

In [2]:
mesh = pymesh.load_mesh("data/brain.ply")

In [ ]:
self_intersection_stats(mesh)

In [ ]:
visualize_intersection(mesh)

#### Save intersection info first

In [3]:
intersections = pymesh.detect_self_intersection(mesh)
intersecting_vertices, intersecting_faces = track_self_intersecting_faces(mesh, intersections)

#### Get outer hull

In [4]:
outer_hull = pymesh.compute_outer_hull(mesh)

In [ ]:
self_intersection_stats(outer_hull)

#### Map intersecting region to modified mesh & Extract submesh

In [5]:
mapped_vertices = map_to_modified_mesh(mesh, outer_hull, intersecting_vertices)

In [6]:
submesh, face_mask = extract_self_intersecting_region_from_modified(outer_hull, mapped_vertices)

In [ ]:
visualize(submesh, filename="Submesh")

In [7]:
remaining_mesh = extract_remaining_mesh(outer_hull, face_mask)

In [ ]:
visualize(remaining_mesh, filename="remaining_mesh")

#### Repair (or Remesh) the submesh
- `pymesh.collapse_short_edges(mesh, abs_threshold, rel_threshold, preserve_feature)`<br>
Collapses all edges with length less than a user specified threshold.

- `pymesh.split_long_edges(mesh, max_edge_length)`<br>
Splits long edges into 2 or more shorter edges.

- `pymesh.remove_obtuse_triangles(mesh, max_angle, max_iterations)`<br>
Splits each obtuse triangle into 2 or more right or sharp triangles.

- `pymesh.remove_degenerated_triangles(mesh, num_iterations)` <br>
Degenerate triangles are triangles with collinear vertices (i.e., some vertices are lying in the same straight line). They have zero areas and their normals are undefined.

In [8]:
from numpy.linalg import norm

def method2(mesh, detail=2.4e-2):

    bbox_min, bbox_max = mesh.bbox
    diag_len = norm(bbox_max - bbox_min)
    target_len = diag_len * detail
    print("Target resolution: {} mm".format(target_len))

    count = 0
    mesh, __ = pymesh.remove_degenerated_triangles(mesh, 100)
    mesh, __ = pymesh.split_long_edges(mesh, target_len)
    num_vertices = mesh.num_vertices
    while True:
        mesh, __ = pymesh.collapse_short_edges(mesh, 1e-6, preserve_feature=True) # Remove extremely small edges
        mesh, __ = pymesh.collapse_short_edges(mesh, target_len,
                                               preserve_feature=True)
        mesh, __ = pymesh.remove_obtuse_triangles(mesh, 150.0, 100)
        if mesh.num_vertices == num_vertices:
            break

        num_vertices = mesh.num_vertices
        print("#v: {}".format(num_vertices))
        count += 1
        if count > 10: break

    mesh = pymesh.resolve_self_intersection(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh = pymesh.compute_outer_hull(mesh)
    mesh, __ = pymesh.remove_duplicated_faces(mesh)
    mesh, __ = pymesh.remove_obtuse_triangles(mesh, 179.9, 5)
    mesh, __ = pymesh.remove_isolated_vertices(mesh)

    return mesh

In [ ]:
components = pymesh.separate_mesh(submesh)

if len(components) == 1:
    repaired_submesh = method2(submesh)
else:
    repaired_components = []
    for compoenent in components:
        repaired_component = method2(compoenent)
        repaired_components.append(repaired_component)
    repaired_submesh = pymesh.merge_meshes(repaired_components)

In [ ]:
visualize(repaired_submesh, filename="Repaired_submesh")

#### Align submesh
Since we have modified the submesh, we need to change its boundary to what it used to be. Otherwise, when we combine the submesh and the remaining component later, there will be some small gaps.

In [10]:
aligned_submesh = align_submesh_boundary(remaining_mesh, repaired_submesh)

In [ ]:
visualize(aligned_submesh, filename="Aligned submesh")

#### Merge two meshes

In [11]:
repaired_full = replace_submesh_in_original(remaining_mesh, aligned_submesh)

In [ ]:
visualize(repaired_full)

#### Final refinement

In [14]:
final = refinement(repaired_full)

In [ ]:
self_intersection_stats(final)

In [ ]:
visualize(final, filename="Final Outerhull")

In [ ]:
evaluation(outer_hull, final)

In [ ]:
evaluate_displacement(outer_hull, final)

In [ ]:
visualize_two_meshes(outer_hull, final)

In [17]:
pymesh.save_mesh("temp.ply", final, ascii=True)